In [1]:
# Modulo para coger informacion de la pokeApi
from api.pokedex import pokedex
# Librerias
import pandas as pd
import os
# Markdown2 convierte una string con codigo markdown a una string HTML
# !pip3 install markdown2
from markdown2 import markdown
# WeasyPrint convierte HTMLs con CSS a PDF
# !pip3 install WeasyPrint
from weasyprint import HTML,CSS

In [2]:
# Elige el nombre o numero del pokemon que quieras para recoger la informacion de la pokeApi
poke_data = pokedex(25)

Abrimos el template markdown que escribimos previamente, poniendo marcadores `{variable}` donde queremos substituir informacion. Eso sirvirá para que podamos hacer un `.format` con esos mismos nombres como keyword arguments para substituirles en el local correcto.

In [3]:
with open("templates/pokereport.md", "r") as file:
    template = file.read()

Empezamos a generar un dicionario con los valores que queremos substituir.

In [4]:
filler = {}
# {pokeball_image}
pokeball = "pokeball" if poke_data["capture_rate"] > 10 else "masterball"
filler["pokeball_image"] = f'![pokeball](images/balls/{pokeball}.png "Pokeball")'
# {name}
filler["name"] = poke_data["evolution_chain"][poke_data["id"]]["name"].title()
# {flavor_text}
filler["flavor_text"] = poke_data["flavor_text"]

In [5]:
# {stats_table}
stats_table = pd.DataFrame.from_dict({pokemon.get("name"):pokemon.get("stats") for pokemon in poke_data["evolution_chain"].values()})
filler["stats_table"] = stats_table.to_markdown()

In [6]:
# {type_table}
poke_types = list(poke_data["evolution_chain"][poke_data["id"]]["types"])
if len(poke_types)<2:
    poke_types.append('')
sep = [":---:"]*len(poke_types)
type_images = [f' ![type](images/types/{typ}.png "{typ}") ' if typ else '' for typ in poke_types]
type_table = "\n".join(["|".join(line) for line in [poke_types,sep,type_images]])
filler["type_table"] = type_table

In [7]:
# {evolution_table}
sprites = [f' ![sprite]({url} "sprite")' for url in [poke["sprite"] for poke in poke_data["evolution_chain"].values()]]
poke_ids = [str(poke_id) for poke_id in poke_data["evolution_chain"].keys()]
if len(sprites)<2:
    sprites.append('')
    poke_ids.append('')
sep = [":---:"]*len(sprites)
evolution_table = "\n".join(["|".join(line) for line in [sprites,sep,poke_ids]])
filler["evolution_table"] = evolution_table

Cuando todo está listo, substituimos esas variables en el template.

In [8]:
output = template.format(**filler)

Finalmente, podemos generar el pdf a partir de el HTML con el CSS para darle estilo.

In [9]:
html = markdown(output, extras=["tables"])
# El argumento `base_url` es para que el interprete de HTML pueda coger bien los ficheros locales, como por 
# ejemplo las imagenes en la carpeta images. Usamos `os.getcwd()` para que el `prefijo` de los paths sea el
# directorio de ese mismo fichero, que sumado a los paths relativos indicado en las imagenes se sumen en la
# ruta que apunta efectivamente a esas imagenes.
html = HTML(string=html, base_url=os.getcwd())
# El fichero CSS, la hoja de estilo, es el fichero que utilizamos para pasar determinadas reglas de formateacion
# al HTML, el tipo o color de las letras, su tamaño, las dimensiones o posición de las imagenes, etc.
# Ambos strings que contienen el codigo HTML o CSS deben ser convertidos a objetos de WeasyPrint.
with open("templates/pokereport.css", "r") as file:
    css = file.read()
css = CSS(string=css)
html.write_pdf(f"output/{filler['name'].lower()}.pdf", stylesheets=[css])

# RESUMEN

* Crear un fichero en Markdown similar a como quiera que sea tu reporte.
* Substituir todas las partes variables del reporte por marcadores `{name}`.
* Crear un dicionário con las keys equivalentes a los marcadores `{name}` del template, asignando a los values, los valores que quieres inserir en esa parte de el reporte.
* Usar el `.format(**filler)` para hacer la substituicion de los marcadores por los valores, generando así una string de codigo Markdown con el documento que deseamos.
* Con `markdown2` y `WeasyPrint`, procesar y generar el pdf en la carpeta `outputs`.